# Решение задачи "Анализ ЭКГ-сигналов для диагностики сердечных патологий" для конкурса AI challenge

Оглавление
- Исследование области задачи
- Обработка и анализ данных
- 

## Исследование области задачи
- SCP-ECG - http://masters.donntu.ru/2008/kita/golovach/library/4_ref/pub.html
- стадии инфаркта

## Обработка и анализ данных

Используемые модули

In [2]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns 

Полезные функции

In [3]:
def get_hr(folder: str, hr_num: str):
    return np.load(f'{folder}/{hr_num}.npy')

Загрузка данных

In [4]:
meta = pd.read_csv('data/train/train_meta.csv')
train_Y = pd.read_csv('data/train/train_gts.csv')

In [89]:
meta.head(3)

,ecg_id,patient_id,age,sex,height,weight,nurse,site,device,recording_date,...,static_noise,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr,record_name,group
0,9,18792.0,55.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-12-08 09:44:43,...,", I-AVR,",NaN,NaN,NaN,NaN,10,records100/00000/00009_lr,records500/00000/00009_hr,00009_hr,1
1,34,13619.0,56.0,0,NaN,NaN,2.0,0.0,CS-12 E,1985-02-10 12:04:03,...,", alles,",NaN,NaN,NaN,NaN,9,records100/00000/00034_lr,records500/00000/00034_hr,00034_hr,3
2,43,11315.0,25.0,1,NaN,63.0,2.0,0.0,CS-12 E,1985-03-14 09:38:53,...,NaN,NaN,NaN,NaN,NaN,8,records100/00000/00043_lr,records500/00000/00043_hr,00043_hr,3


In [ ]:
train_Y.head(3)

### Анализ

Просмотрим инофрмацию о нашем датасете </br>
https://physionet.org/content/ptb-xl/1.0.3/ - здесь тоже можно

In [5]:
meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2101 entries, 0 to 2100
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ecg_id                        2101 non-null   int64  
 1   patient_id                    2101 non-null   float64
 2   age                           2101 non-null   float64
 3   sex                           2101 non-null   int64  
 4   height                        689 non-null    float64
 5   weight                        905 non-null    float64
 6   nurse                         1947 non-null   float64
 7   site                          2100 non-null   float64
 8   device                        2101 non-null   object 
 9   recording_date                2101 non-null   object 
 10  report                        2101 non-null   object 
 11  scp_codes                     2101 non-null   object 
 12  heart_axis                    1231 non-null   object 
 13  inf

Удалим те данные, которые не несут важной для нас информации

In [6]:
meta = meta.drop(columns=['ecg_id', 'patient_id', 'filename_lr', 'filename_hr', 'recording_date', 'nurse', 'site'])

Рассмотрим те столбцы, в которых есть много пропусков

In [8]:
meta['site'].value_counts()

site
0.0     810
1.0     618
2.0     518
3.0      61
4.0       9
5.0       9
7.0       7
10.0      6
11.0      5
16.0      5
27.0      4
15.0      3
21.0      3
18.0      3
17.0      3
8.0       3
6.0       3
9.0       3
25.0      2
31.0      2
22.0      2
32.0      2
29.0      2
14.0      2
19.0      2
44.0      1
12.0      1
41.0      1
43.0      1
42.0      1
26.0      1
30.0      1
23.0      1
35.0      1
47.0      1
28.0      1
20.0      1
38.0      1
Name: count, dtype: int64

In [7]:
empty_columns = ['height', 'weight', ] # Придется смотреть вручную, т.к. у некоторых столбцов Nan - просто отсутствие, а не пропуск

for column in meta.columns:
    print(f'Column name: {column}', 'HAVE MANY EMPTY VALUES' if column in empty_columns else '')
    print(meta[column].value_counts() if len(meta[column].unique()) < 14 else f'so much unique values\n{meta[column].describe()}')
    print()

Column name: age 
so much unique values
count    2101.000000
mean       62.581152
std        31.758759
min         3.000000
25%        50.000000
50%        61.000000
75%        72.000000
max       300.000000
Name: age, dtype: float64

Column name: sex 
sex
0    1095
1    1006
Name: count, dtype: int64

Column name: height HAVE MANY EMPTY VALUES
so much unique values
count    689.000000
mean     167.343977
std       10.479595
min      100.000000
25%      160.000000
50%      167.000000
75%      175.000000
max      209.000000
Name: height, dtype: float64

Column name: weight HAVE MANY EMPTY VALUES
so much unique values
count    905.000000
mean      71.605525
std       15.397469
min       16.000000
25%       61.000000
50%       70.000000
75%       80.000000
max      180.000000
Name: weight, dtype: float64

Column name: site 
so much unique values
count    2100.000000
mean        1.583333
std         3.964613
min         0.000000
25%         0.000000
50%         1.000000
75%         2.00000

Заменим строки в названиях, так как они малину портят

In [ ]:
str_columns = [] #TODO ONLY SOME COLUMNS

meta_test = pd.get_dummies(meta, columns=str_columns)